# **Dependências do Projeto**

In [1]:
%pip install -U selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install torch torchvision torchaudio
#Instalando o pytorch

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
#Caso não consiga pydirectinput
%pip install pyautogui

Note: you may need to restart the kernel to use updated packages.


In [5]:
#Biblioteca de aprendizado por reforço
%pip install stable-baselines3[extra] protobuf==3.20.*

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install mss pydirectinput pytesseract

Note: you may need to restart the kernel to use updated packages.


# **Imports**

In [1]:
#Para captura de tela
from mss import mss
#import pyautogui #para comandos de teclado
import cv2 
#Tratamentos dos frames
import numpy as np 
import pyautogui
#Framework intermediário paara trabalhar com as imagens
#import pytesseract #para extrair o game over da imagem
from matplotlib import pyplot as plt #Visualizando resultados
import time
from PIL import Image
#Coisas para construir o ambinete
from gym import Env
from gym.spaces import Box, Discrete, Dict
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from io import BytesIO
from PIL import Image
from selenium.common.exceptions import WebDriverException
import base64
from io import BytesIO


In [2]:
%load_ext tensorboard

# **Construindo o Ambiente**

In [3]:
from collections import deque

In [4]:
#Classe para definir o ambiente
class WebGame(Env):
    #ESSAS FUNÇÃO SÃO NECESSÁRIAS POR CONTA DO GYM
    #Função responsavel pela inicialização do ambiente;
    #Portanto onde ficam todas as variaveis do ambiente, configuração das ações e da observação 
    def __init__(self):
        super().__init__()

        #Configuração do tempo
        self.print_time = .005
        self.wait_time = .015
        
        #Configuração do espaço
        self.observation_space = Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8)
        #self.observation_space = Dict({
        #    "print":Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8),
        #    "print2":Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8),
        #    "print3":Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8),
            #"print4":Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8),
        #    "score":Box(low=0, high=10000, shape=(1,), dtype=np.uint8),
        #                              })
        self.action_space = Discrete(3)

        #Variavei para a captura de tela
        self.screen_cap = mss() #Lib de captura de tela
        self.game_location = {"top": 300, "left": 0, "width": 650, "height": 500} #Área do monitor com 
        self.game_done = {"top": 40, "left": 0, "width": 800, "height": 640}

        self.chromedriver_path = "./chromedriver.exe"
        _chrome_options = webdriver.ChromeOptions()
        _chrome_options.add_argument("--mute-audio")
        _chrome_options.add_argument("--disable-gpu")
        self._driver = webdriver.Chrome(executable_path=self.chromedriver_path,options=_chrome_options)

        self.first_image = np.zeros(5)
        self.last_image = np.zeros((500,650,3))
        
        #Dicionário de ações
        self.action_dict = {
            0:Keys.ARROW_DOWN,
            1:Keys.ARROW_UP,
            2:'nothing'
        }

    #Função responsavel por passar as ações pro jogo, para fazer algo pro jogo
    def step(self, action):
        # Se ação escolhida for um dos botões, realizar a ação.
        if action != 2:
            self._driver.find_elements(By.TAG_NAME, "body")[0].send_keys(self.action_dict[int(action)])
        
        #Checa a próxima ação
        observation = self.get_observation()
        #Todas as ações tem que verificar se a ação acabou
        done, done_cap = self.get_done() 
        
        #Pegando o score do jogo
        score = self.get_score()
        #Ganhamos 1 ponto por cada frame que estamos vivos.
        reward = 1
        #É um dicionário de informações que retornam através do que a gente precisa
        info = {
            'score':score
        }

        time.sleep(self.wait_time)
        
        return observation, reward, done, info
        

    #Função para a visualização do jogo:
    def render(self, mode: str='human'):
        img = cv2.cvtColor(self._get_image(), cv2.COLOR_BGR2RGB)
        if mode == 'rgb_array':
            return img
        elif mode == 'human':
            from gym.envs.classic_control import rendering
            if self.viewer is None:
                self.viewer = rendering.SimpleImageViewer()
            self.viewer.imshow(img)
            return self.viewer.isopen

        # cv2.imshow('Game', np.array(self.cap.grab(self.game_location))[:,:,:3])
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     self.close()

    #Função responsavel por recomeçar o jogo
    def reset(self):
        #Try catch precisa existir pq toda a vez q o selenium detecta que ele viu o chrome dino,
        #ele dispara um erro que esta sem internet.
        try:
            self._driver.get('chrome://dino')
            
        except WebDriverException:
            pass
        WebDriverWait(self._driver, 10).until(
            EC.presence_of_element_located((
                By.CLASS_NAME, 
                "runner-canvas"
            ))
        )
        self._driver.implicitly_wait(0.5)
        self._driver.find_elements(By.TAG_NAME, "body")[0].send_keys(Keys.ARROW_UP)
        

        return self.get_observation()

    #FUNÇÕES CUSTOM
    #Fecha a parte de visualização
    def close(self):
        cv2.destroyAllWindows()
    
    def get_score(self):
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array)
        #shape = np.zeros(1)
        #shape[0] = int(score)
        return int(score)
    
    def get_score_shape(self):
        shape = np.zeros(1)
        shape[0] = int(self.get_score())
        return shape
    
    def get_img(self):
        LEADING_TEXT = "data:image/png;base64,"
        canvas = self._driver.execute_script("return document.querySelector('canvas.runner-canvas').toDataURL()")
        img = canvas[len(LEADING_TEXT):]
        img_data = np.array(Image.open(BytesIO(base64.b64decode(img))))
        return img_data 
        # img = self._driver.execute_script()
    
    #Função responsavel por pegar a imagem do jogo e tratar da forma necessária.
    def get_print(self): 

        #img = numpy.array(self.screen_cap.grab(game_location))[:,:,3].astype(np.uint8)
        raw = np.array(self.get_img())[:,:,:3].astype(np.uint8)
        img = raw[:200,:400]
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (200,83))
        channel = np.reshape(resized, (1,83,200))
        
        self.first_image = self.last_image
        self.last_image = raw
        
        return channel
    
    #Função responsavel por pegar a imagem do jogo e tratar da forma necessária.
    def get_observation(self): 
        #print1 = self.get_print()
        #self._driver.implicitly_wait(self.print_time)
        #print2 = self.get_print()
        #self._driver.implicitly_wait(self.print_time)
        #return {"print":print1, "print2":print2, "print3":self.get_print(), "score":self.get_score_shape() }
        return self.get_print()


    #Função para pegar o texto de fim de jogo:
    def get_done(self):
        #Se as duas imagens iniciais forem iguais acabou jogo
        done = False
        #Colando a primeira condição devido ao fator de que as formas diferentes estavam dando problema na comparação devido ao np.all
        
        if self.first_image.shape != self.last_image.shape:
            return done, self.last_image

        if np.all(np.equal(self.first_image, self.last_image)) and not self._driver.execute_script("return Runner.instance_.playing"):
            done = True
        
        return done, self.last_image

# Treinamento

In [5]:
# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback, CheckpointCallback, CallbackList
# Verificando se o ambiente é válido para fazer as coisa
from stable_baselines3.common import env_checker

from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, SubprocVecEnv

In [6]:
from stable_baselines3 import DQN, PPO, DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack,SubprocVecEnv
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.logger import Figure, configure

In [7]:
env_lambda = lambda: WebGame()
env2 = SubprocVecEnv([env_lambda for i in range(4)])
#env2 = WebGame()

In [8]:
env2.reset()

array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]]], dtype=uint8)

In [9]:
#Criandos as pastas para que funcione.
CHECKPOINT_DIR = './checkpoints/'
LOG_DIR = './logs/'

In [10]:
checkpoint_callback = CheckpointCallback(
  save_freq=1000,
  save_path=CHECKPOINT_DIR,
  name_prefix="dqn_single_multi_model",
  save_replay_buffer=False,
  save_vecnormalize=True,
    verbose=2
)
eval_callback = EvalCallback(env2, best_model_save_path=CHECKPOINT_DIR,
                             log_path=LOG_DIR, eval_freq=1000,
                             deterministic=True, render=False)
callback = CallbackList([checkpoint_callback, eval_callback])

In [11]:
def get_model():
    #model = PPO('MultiInputPolicy', env2, tensorboard_log=LOG_DIR, learning_rate=0.005, verbose=2, policy_kwargs=dict(normalize_images=False))
    model = DQN('CnnPolicy', env2, tensorboard_log=LOG_DIR, verbose=2, buffer_size=12000, learning_rate=0.0006, 
        learning_starts=500, exploration_fraction=0.4, exploration_initial_eps=0.9, exploration_final_eps=0.05,
                policy_kwargs=dict(normalize_images=False))
    return model

In [12]:
model = get_model()

Using cpu device


In [13]:
#Começa o treinamento
model.learn(total_timesteps=800000, callback=callback)

Logging to ./logs/DQN_28
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.899    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 66       |
|    time_elapsed     | 6        |
|    total_timesteps  | 464      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.898    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 59       |
|    time_elapsed     | 13       |
|    total_timesteps  | 772      |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.033    |
|    n_updates        | 17       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.897    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 55    

E:\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=4000, episode_reward=90.20 +/- 2.64
Episode length: 90.20 +/- 2.64
----------------------------------
| eval/               |          |
|    mean_ep_length   | 90.2     |
|    mean_reward      | 90.2     |
| rollout/            |          |
|    exploration_rate | 0.889    |
| time/               |          |
|    total_timesteps  | 4000     |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0268   |
|    n_updates        | 218      |
----------------------------------
New best mean reward!
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.889    |
| time/               |          |
|    episodes         | 52       |
|    fps              | 44       |
|    time_elapsed     | 90       |
|    total_timesteps  | 4012     |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0171   |
|    n_updates        | 219      |
-----------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.877    |
| time/               |          |
|    episodes         | 116      |
|    fps              | 43       |
|    time_elapsed     | 203      |
|    total_timesteps  | 8804     |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0133   |
|    n_updates        | 519      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.876    |
| time/               |          |
|    episodes         | 120      |
|    fps              | 43       |
|    time_elapsed     | 208      |
|    total_timesteps  | 9060     |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.02     |
|    n_updates        | 535      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.863    |
| time/               |          |
|    episodes         | 184      |
|    fps              | 43       |
|    time_elapsed     | 319      |
|    total_timesteps  | 13924    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0106   |
|    n_updates        | 839      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.862    |
| time/               |          |
|    episodes         | 188      |
|    fps              | 43       |
|    time_elapsed     | 324      |
|    total_timesteps  | 14184    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0095   |
|    n_updates        | 855      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

...

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9940     |
|    fps              | 41       |
|    time_elapsed     | 18638    |
|    total_timesteps  | 778936   |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 1.05     |
|    n_updates        | 48652    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9944     |
|    fps              | 41       |
|    time_elapsed     | 18645    |
|    total_timesteps  | 779272   |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.816    |
|    n_updates        | 48673    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 10004    |
|    fps              | 41       |
|    time_elapsed     | 18776    |
|    total_timesteps  | 784424   |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 2.08     |
|    n_updates        | 48995    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 10008    |
|    fps              | 41       |
|    time_elapsed     | 18783    |
|    total_timesteps  | 784732   |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 1.13     |
|    n_updates        | 49014    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 10072    |
|    fps              | 41       |
|    time_elapsed     | 18903    |
|    total_timesteps  | 789796   |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 2.07     |
|    n_updates        | 49331    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 10076    |
|    fps              | 41       |
|    time_elapsed     | 18909    |
|    total_timesteps  | 790048   |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 1.29     |
|    n_updates        | 49346    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 10140    |
|    fps              | 41       |
|    time_elapsed     | 19019    |
|    total_timesteps  | 794696   |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.896    |
|    n_updates        | 49637    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 10144    |
|    fps              | 41       |
|    time_elapsed     | 19026    |
|    total_timesteps  | 794968   |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 1.54     |
|    n_updates        | 49654    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 10208    |
|    fps              | 41       |
|    time_elapsed     | 19137    |
|    total_timesteps  | 799768   |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 1.33     |
|    n_updates        | 49954    |
----------------------------------
Saving model checkpoint to ./checkpoints/dqn_single_multi_model_800000_steps.zip
Eval num_timesteps=800000, episode_reward=90.20 +/- 2.23
Episode length: 90.20 +/- 2.23
----------------------------------
| eval/               |          |
|    mean_ep_length   | 90.2     |
|    mean_reward      | 90.2     |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 800000   |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.

# Teste

In [15]:
env2 = WebGame()
model = get_model()
model.load('checkpoints/best_model') 

for episode in range(5): 
    obs = env2.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env2.step(action)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))

    

C:\Users\Dan\AppData\Local\Temp\ipykernel_16964\830487098.py:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self._driver = webdriver.Chrome(executable_path=self.chromedriver_path,options=_chrome_options)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Total Reward for episode 0 is 121
Total Reward for episode 1 is 111
Total Reward for episode 2 is 45
Total Reward for episode 3 is 111
Total Reward for episode 4 is 112


In [14]:
%tensorboard --logdir ./logs/DQN_28/ --host localhost